In [25]:
import pymongo
from pprint import pprint
from datetime import datetime

In [26]:
def utc_to_dt(utc: float) -> datetime:
    """Convert a unix time to a python datetime."""
    return datetime.utcfromtimestamp(int(utc))

In [6]:
# define mongo connection
conn = pymongo.MongoClient('localhost', 27017)
db = conn["drug_pricing"]
coll = db["praw"]

In [52]:
q = coll.find({ "$text": {"$search": "lawrence november 2019"} })

In [53]:
print(q[0]["text"])

I am a bit of a science nerd, on an amatuer basis, and like reading articles related to tech and addiction.

So this morning, my favorite tech site is talking about gene editing and the state of the art in fixing your crappy genes. There is a technique called CRISPR that scientists now have available. If I am getting this right, they can kind of cut and paste genes now. They use enzymes to take out bad dna and replace it with better, improved versions.

Then, to get the new dna into the body, they take a common virus, put the new dna inside, and inject it into a mouse to infect the little guy with better genetic material.

The idea is that the new dna will replace whatever bad gene is causing the issues.

Seems a good fit for addiction. Right? Just find whatever gene causes you to do dope, cut it out and replace it with normal person dna.

Well, not quite that simple. Yet. First, it is not for sure that addiction is caused by genes. Current thinking in the US is that it is a 50 - 50 de

In [54]:
str(utc_to_dt(1271701857))

'2010-04-19 18:30:57'